# **Module 4: Diagnostic Systems**
## DAT410

### Group 29 
### David Laessker, 980511-5012, laessker@chalmers.se

### Oskar Palmgren, 010529-4714, oskarpal@chalmers.se



We hereby declare that we have both actively participated in solving every exercise. All solutions are entirely our own work, without having taken part of other solutions.

___


## 1) Article summary and take-aways

___
## 2) Implementation

In [ ]:
import pandas as pd

# load dataset as a df
data = pd.read_pickle('wdbc.pkl')

# show the first few rows
data.head()

___
## 3) Disucussion